In [1]:
import sys
import numpy as np
from datetime import datetime

In [2]:
class state():
    def __init__(self, matrix, parent=None, move=None):
        self.matrix      = np.array(matrix)
        self.parent      = parent           # The parent to this state. The start state will not have any parrent
        self.move        = move             # What move (Left/Right/Up/Down) has led to this state
        self.max_row     = len(matrix)      # Max rows in the matrix
        self.max_column  = len(matrix[0])   # Max columns in the matrix
        self.hash        = self.get_hash()  # Has of the matrix. Will be used while comparing states.
        self.depth       = self.set_depth() # The depth of this state from the state_state

    
    def get_value(self, i, j):
        ''' Get the value from mattrix at [i][j] location '''
        return self.matrix[i, j]
    
    def get_i_j_of(self, val):
        '''Get the i & j of the value in the matrix '''
        return np.where(self.matrix == val)
    
    def get_copy(self):
        ''' Returns the copy of current matrix '''
        return self.matrix.copy()
    
    def set_depth(self):
        ''' Set the depth of this state. The depth will be parent's depth + 1 '''
        if self.parent is not None:
            return self.parent.get_depth() + 1
        else:
            return 0

    def get_depth(self):
        ''' Get function to get the depth of the state '''
        return self.depth
    
    def get_hash(self):
        '''
        Let us create the hash of the matrix so that the we will add the hash of explored state
        in the explored list. Before processing any state, we will check if that state us already
        explored or not.
        To calculate the hash, we are makeing int of the matrix as below
        [[123]
         [456]
         [780]] = 1234567890
        So, every matrix will have a unique hash value
        '''
        multiplier = 100000000
        value = 0
        for i in range(self.max_row):
            for j in range (self.max_column):
                value += self.matrix[i, j] * multiplier
                multiplier //= 10
        return value
    
    def print(self):
        ''' print the matrix '''
        for i in range (self.max_row):
            for j in range (self.max_column):
                val = self.matrix[i, j]
                if val == 0:
                    print("B", end=" ")
                else:
                    print(val, end=" ")
            print(" ")

In [3]:
class Queue():
    def __init__(self):
        self.list = []
        self.max_size = 0

    def current_size(self):
        return len(self.list)
    
    def max_grown_size(self):
        return self.max_size
    
    def empty(self):
        return self.current_size() == 0

    def add(self, item):
        self.list.append(item)
        self.max_size = max(self.max_size, self.current_size())   

    def remove(self):
        if self.current_size() == 0:
            raise Exception("Queue is already empty")
        else:
            return self.list.pop(0)

In [4]:
class Stack(Queue):

    def remove(self):
        if self.current_size() == 0:
            raise Exception("Queue is already empty")
        else:
            return self.list.pop(-1)

In [5]:
class puzzle():
    def __init__(self, start, goal, max_depth):
        # Check if the nput Start state and Goal state are valid or not.
        self.validate_input_matrix(start)
        self.validate_input_matrix(goal)
                
        self.start_state   = state(start)
        self.goal_state    = state(goal)
        self.max_depth     = max_depth
        self.max_row, self.max_column = start.shape
        self.explored_list = []
        self.solution_steps = []
        self.start_time     = datetime.now()
        self.end_time       = datetime.now()
        
        print(f"Start state:")
        self.start_state.print()
        
        print(f"\nGoal state:")
        self.goal_state.print()
        
    
    def validate_input_matrix(self, matrix):
        ''' Validate the matrix. The matrix should have 0-8 values and no value should be repeated.
            While creating the puzzle object, validation should be done on start_state & goal_state '''
        max_row = len(matrix)
        max_column = len(matrix[0])
        
        for value in range(0, max_row*max_column):
            found = False;
            for i in range(max_row):
                for j in range(max_column):
                    if(value == matrix[i, j]):
                        found = True
            
            if(found == False):
                raise Exception("This is not a valid matrix")

        
    def get_neighbor_states(self, current_state):
        ''' Get the neighbors of the current state. The neighbors will be the states that comes
            after taking Left/Right/Up/Down move.
            No validation is done on the neighbor in this function.
            The called of this function need to do all the required validations'''
        
        move_directions = [("LEFT", 0, -1), ("RIGHT", 0, 1), ("UP", -1, 0), ("DOWN", 1, 0)]
        row, col = current_state.get_i_j_of(B)
        neighbors = []
        
        # if depth is configured then check the depth. If the depth is more then return empty list to the caller 
        if self.max_depth > 0 and current_state.get_depth() >= self.max_depth:
            return neighbors
        
        for move, i_offset, j_offset in move_directions:
            new_i = row + i_offset
            new_j = col + j_offset
            
            # move the blank space Left/Right/Up/Down if it is not crossing the boundries
            if ( 0 <= new_i < self.max_row ) and ( 0 <= new_j < self.max_column) :
                new_state = self.move(current_state, move, row[0], col[0])
                neighbors.append(new_state)

        return neighbors


    def move(self, current_state, move, i, j):
        '''MOve the blank space to Left/Right/Up/Down and return the new state
           The current state will be parent to tthis new state'''
        temp_matrix = current_state.get_copy()
        
        # Store the value of matrix[i, j] in temp varianble. This will be used during swap operation
        temp_val = temp_matrix[i, j]
        
        if move == "RIGHT":
            temp_matrix[i, j] = temp_matrix[i, j+1]
            temp_matrix[i, j+1] = temp_val
        elif move == "LEFT":
            temp_matrix[i, j] = temp_matrix[i, j-1]
            temp_matrix[i, j-1] = temp_val
        elif move == "UP":
            temp_matrix[i, j] = temp_matrix[i-1, j]
            temp_matrix[i-1, j] = temp_val
        elif move == "DOWN":
            temp_matrix[i, j] = temp_matrix[i+1, j]
            temp_matrix[i+1, j] = temp_val

        # Create the state from this new matrix. Assign the current matrix as parent to this new state
        new_state = state(temp_matrix, current_state, move)
        return new_state


    def print_solution(self):
        ''' Print the solution '''
        while len(self.solution_steps) != 0:
            current_state = self.solution_steps.pop(0)
            print("\nMove: ", current_state.move)
            current_state.print()

    
    def solve_using(self, type_of_algorithm, print_sol, max_states_to_be_checked):
        
        # valdate the type of algorithm that is given. If it is BFS/DFS the init the Queue/stack respectively
        if (type_of_algorithm == "BFS"):
            states_to_be_explored = Queue()
            type_of_list="Queue"                
        elif (type_of_algorithm == "DFS"):
            states_to_be_explored = Stack()
            type_of_list="Stack"
        else:
            print(f"{type_of_algorithm} is not implemented. Try with other type of algorithm")
            return
        
        # Clear the explored list, solution steps
        self.explored_list.clear()
        self.solution_steps.clear()

        self.start_time = datetime.now()
        no_of_states_explores = 0
        
        print(f"\nStarted solving puzzle using {type_of_algorithm} at {self.start_time}")

        states_to_be_explored.add(self.start_state)
        self.explored_list.append(self.start_state.hash)
        
        # Keep looping until solution is not found
        while not states_to_be_explored.empty():
            
            # Remove a state from the Queue/Stack
            current_state = states_to_be_explored.remove()
            no_of_states_explores += 1
            
            if((max_states_to_be_checked > 0) and  (max_states_to_be_checked < no_of_states_explores)):
                print(f"{type_of_algorithm}: Exiting... Explored configured # of states: {max_states_to_be_checked}");
                break
            
            # if hash of current state and Goal state is same then we have reached the goal :D
            if current_state.hash == self.goal_state.hash:
                self.end_time = datetime.now()
                execution_time = self.end_time - self.start_time
                print(f"{type_of_algorithm}: {self.end_time}")    
                print(f"{type_of_algorithm}: Solution found after exploring states: {no_of_states_explores}")
                
                
                item = current_state
                while item.parent is not None:
                    self.solution_steps.insert(0, item)
                    item = item.parent
                
                print(f"{type_of_algorithm}: Solution can be reached in steps: {len(self.solution_steps)}")
                print(f"{type_of_algorithm}: Time taken to find the solution: {execution_time}")
                print(f"{type_of_algorithm}: Goal state found at depth: {current_state.get_depth()} ")
                print(f"{type_of_algorithm}: {type_of_list} has grown max up to: {states_to_be_explored.max_grown_size()}")
                
                if print_sol == True :
                    print("\nStart state: ")
                    self.start_state.print()
                    
                    print("\nGoal state: ")
                    self.goal_state.print()
        
                    print("\nSolution steps: \n---------------------")
                    self.print_solution()
                
                return
            
            # get the valid neighbors of current state. Drop the neighbor that is already explored 
            neighbors = self.get_neighbor_states(current_state)
            for neighbor in neighbors:
                if neighbor.hash not in self.explored_list:
                    self.explored_list.append(neighbor.hash)
                    states_to_be_explored.add(neighbor)
                    
        #############     End while Loop      ####################
        
        self.end_time = datetime.now()
        execution_time = self.end_time - self.start_time
        
        print(f"\n{type_of_algorithm}: {self.end_time}")
        print(f"{type_of_algorithm}: No solution found")
        print(f"{type_of_algorithm}: Time consumed: {execution_time}")
        print(f"{type_of_algorithm}: States explored {no_of_states_explores}")
        print(f"{type_of_algorithm}: Checked upto depth: {current_state.get_depth()} Vs configured {self.max_depth}" )
        print(f"{type_of_algorithm}: {type_of_list} has grown max up to: {states_to_be_explored.max_grown_size()}")
        

In [11]:
print("============================================================================")
print("This code is part of Assignment 1 of CS561 - Executive-Assignment-1 Lab")
print("Submitted by:")
print("\tSukhvinder Singh (admission No: IITP001300) (email id: sukhvinder.malik13@gmail.com)")
print("\tManjit Singh Duhan (admission No: IITP001316) (email id: duhan.manjit@gmail.com)")
print("============================================================================\n\n")


B=0  # Just to make the matrix more readable as eyes will easily catch "B" compared to 0.

#take a random array
random_array = np.arange(9)
np.random.shuffle(random_array)

# enter the start state manually. (ToDo: If requirement is to generate random matrix, then neew to write a funtion for that)
start_state = random_array.reshape(3, 3)

# The goal state. This should be the goal where we need to reach
goal_state= np.array([
       [1, 2, 3],
       [4, 5, 6],
       [7, 8, B]])

# Enabled if want to see the solution
print_solution = False

# Configure the max depth. 0 for unlimited
max_depth = 0

# configure the max states to be checked. 0 for unlimited
max_states_to_be_checked = 0

# configure the type of algorithm. Currently "BFS" & "DFS" are supported
type_of_algorithm = "BFS"

p = puzzle(start_state, goal_state, max_depth)
p.solve_using(type_of_algorithm, print_solution, max_states_to_be_checked)


type_of_algorithm = "DFS"
p.solve_using(type_of_algorithm, print_solution, max_states_to_be_checked)

This code is part of Assignment 1 of CS561 - Executive-Assignment-1 Lab
Submitted by:
	Sukhvinder Singh (admission No: IITP001300) (email id: sukhvinder.malik13@gmail.com)
	Manjit Singh Duhan (admission No: IITP001316) (email id: duhan.manjit@gmail.com)


Start state:
1 6 8  
4 3 2  
5 7 B  

Goal state:
1 2 3  
4 5 6  
7 8 B  

Started solving puzzle using BFS at 2023-08-19 15:28:32.039768
BFS: 2023-08-19 15:31:42.057852
BFS: Solution found after exploring states: 54702
BFS: Solution can be reached in steps: 20
BFS: Time taken to find the solution: 0:03:10.018084
BFS: Goal state found at depth: 20 
BFS: Queue has grown max up to: 17289

Started solving puzzle using DFS at 2023-08-19 15:31:42.096855


KeyboardInterrupt: 